In [2]:
import torch
from collections import Counter
import math
from gensim.models import Word2Vec
import transformers

## Sparse Embedding

### One-hot Encoding

In [3]:
def one_hot_encoding(labels, num_classes):
    return torch.nn.functional.one_hot(labels, num_classes)

labels = torch.tensor([0, 1, 2, 1])
num_classes = 3
encoder_labels =  one_hot_encoding(labels, 3)
encoder_labels

tensor([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 1, 0]])

In [4]:
data = "táo thơm chuối táo ớt táo ớt"
unique_values = list(set(data.split()))  

value_to_index = {value: idx for idx, value in enumerate(unique_values)}
encoded_data = torch.tensor([value_to_index[item] for item in data.split()])
encoded_data, value_to_index

(tensor([3, 1, 0, 3, 2, 3, 2]), {'chuối': 0, 'thơm': 1, 'ớt': 2, 'táo': 3})

### Bad-of-Words (BoW)

In [5]:
corpus = [
    "táo thơm ớt ớt thơm",
    "chuối táo thơm",
    "chuối ớt táo chuối"
]

vocab = set()
for sentence in corpus:
    vocab.update(sentence.split())

vocab = sorted(list(vocab))
word_to_index = {word: idx for idx, word in enumerate(vocab)}

print(f"Từ điển (Vocabulary): {word_to_index}")


def bow_vector(sentence, word_to_index):
    # Tạo vector ban đầu với kích thước bằng số từ trong từ điển
    bow_vector = torch.zeros(len(word_to_index), dtype=torch.int)
    
    # Tăng số lần xuất hiện của từ trong câu
    for word in sentence.split():
        if word in word_to_index:
            idx = word_to_index[word]
            bow_vector[idx] += 1
    return bow_vector

bow_vectors = [bow_vector(sentence, word_to_index) for sentence in corpus]

for i, vector in enumerate(bow_vectors):
    print(f"Câu {i+1}: {corpus[i]}")
    print(f"BoW vector: {vector}")


Từ điển (Vocabulary): {'chuối': 0, 'thơm': 1, 'táo': 2, 'ớt': 3}
Câu 1: táo thơm ớt ớt thơm
BoW vector: tensor([0, 2, 1, 2], dtype=torch.int32)
Câu 2: chuối táo thơm
BoW vector: tensor([1, 1, 1, 0], dtype=torch.int32)
Câu 3: chuối ớt táo chuối
BoW vector: tensor([2, 0, 1, 1], dtype=torch.int32)


### TF-IDF (Term Frequency-Inverse Document Frequency)

In [6]:
vocab = set()
for sentence in corpus:
    vocab.update(sentence.split())

vocab = sorted(list(vocab))
word_to_index = {word: idx for idx, word in enumerate(vocab)}

def compute_tf(sentence, word_to_index):
    tf = torch.zeros(len(word_to_index), dtype=torch.float32)
    word_count = len(sentence.split())
    
    word_freq = Counter(sentence.split())
    for word, freq in word_freq.items():
        idx = word_to_index[word]
        tf[idx] = freq / word_count
    return tf

def compute_idf(corpus, word_to_index):
    N = len(corpus)
    idf = torch.zeros(len(word_to_index), dtype=torch.float32)
    
    for word, idx in word_to_index.items():
        # Đếm số văn bản chứa từ 'word'
        doc_count = sum(1 for sentence in corpus if word in sentence.split())
        idf[idx] = math.log((N + 1) / (doc_count + 1)) + 1  # Sử dụng +1 để tránh log(0)
    
    return idf

# Tính TF cho mỗi câu trong corpus
tf_vectors = [compute_tf(sentence, word_to_index) for sentence in corpus]

# Tính IDF cho toàn bộ corpus
idf_vector = compute_idf(corpus, word_to_index)

# Bước 4: Tính TF-IDF
tf_idf_vectors = [tf * idf_vector for tf in tf_vectors]

for i, tf_idf in enumerate(tf_idf_vectors):
    print(f"Câu {i+1}: {corpus[i]}")
    print(f"TF-IDF vector: {tf_idf}")

Câu 1: táo thơm ớt ớt thơm
TF-IDF vector: tensor([0.0000, 0.5151, 0.2000, 0.5151])
Câu 2: chuối táo thơm
TF-IDF vector: tensor([0.4292, 0.4292, 0.3333, 0.0000])
Câu 3: chuối ớt táo chuối
TF-IDF vector: tensor([0.6438, 0.0000, 0.2500, 0.3219])


## Dense Vector Embedding

### Word2Vec

In [7]:
from gensim.models import Word2Vec

# Dữ liệu đầu vào (đã tách từ)
corpus = [
    ["táo", "thơm", "ớt", "bút", "apple"],
    ["chuối", "táo", "thơm", "sách"],
    ["chuối", "ớt", "táo", "vở"]
]

# Huấn luyện mô hình
model = Word2Vec(sentences=corpus, vector_size=50, window=2, min_count=1, sg=1, epochs=100)

# Lấy vector của từ "táo"
print("Vector của 'táo':", model.wv["táo"])

# Tìm từ tương tự
print("Các từ giống 'táo':", model.wv.most_similar("táo"))


Vector của 'táo': [-1.13135285e-03  5.20269095e-04  1.01734819e-02  1.80193000e-02
 -1.85464732e-02 -1.42590366e-02  1.29538365e-02  1.79792605e-02
 -1.00492900e-02 -7.50996405e-03  1.47669278e-02 -3.06831114e-03
 -9.06716380e-03  1.30750434e-02 -9.67771932e-03 -3.62855755e-03
  5.80543466e-03  2.03715661e-03 -1.66312736e-02 -1.88968182e-02
  1.46523351e-02  1.01409890e-02  1.35517530e-02  1.49652897e-03
  1.27245337e-02 -6.82014087e-03 -1.85294356e-03  1.15764104e-02
 -1.50756668e-02 -7.88858440e-03 -1.50152389e-02 -1.87121518e-03
  1.90807041e-02 -1.46380691e-02 -4.69329720e-03 -3.85107379e-03
  1.62358470e-02 -1.18842153e-02  7.62698182e-05 -9.45923291e-03
 -1.92045942e-02  1.00192167e-02 -1.75131075e-02 -8.79179314e-03
  3.78884988e-06 -5.36486798e-04 -1.53277135e-02  1.91788729e-02
  9.94655956e-03  1.84892565e-02]
Các từ giống 'táo': [('bút', 0.13324227929115295), ('ớt', 0.1281765252351761), ('chuối', 0.0448036827147007), ('sách', 0.013010905124247074), ('thơm', -0.01318899542093

### ELMo (Contextual Embedding)

### FastText

In [9]:
from gensim.models import FastText

# Tạo model FastText
sentences = [["i", "like", "apple"], ["i", "like", "banana"], ["apple", "banana", "orange"]]
model = FastText(sentences, min_count=1)

# Lấy vector cho một từ
word_vector = model.wv['apple']
print(word_vector)


[ 4.00545076e-04  2.36457260e-03  1.66398095e-04 -2.81718210e-04
  1.03331089e-03 -4.12005320e-04  3.38438689e-03 -7.33376306e-04
  2.20219765e-04 -1.12564536e-03  7.10081309e-04 -6.44268701e-04
 -3.60235077e-04 -2.58245622e-04  1.90043473e-03 -3.37280682e-04
  8.52563593e-04  1.09980674e-03  2.16988381e-03  1.68166670e-03
 -8.98101018e-04 -9.09217633e-04  6.65286439e-04 -1.85854291e-03
  7.52204389e-04 -1.11214141e-03 -2.41464889e-03  1.78417715e-03
  1.36162445e-03 -9.08752321e-04 -7.20086173e-05  1.94867112e-04
 -8.43438378e-04 -1.04499399e-03 -1.08950108e-03 -1.53674380e-04
  4.64575103e-04  2.29662028e-03  2.46573414e-04 -1.57911761e-03
 -1.24617736e-03  6.32994110e-04 -6.77781703e-04 -4.15293500e-04
 -1.88943720e-03  8.19688430e-04  1.23914925e-03  1.68748476e-04
 -3.71455972e-04 -1.18097255e-03  2.91015883e-03  1.09555596e-03
 -2.39176350e-03  1.03498471e-03 -8.77617160e-04  4.00312897e-03
 -5.67850540e-04 -2.16231419e-04  2.67395470e-03  9.31102608e-04
  2.44583003e-04  9.70419

## Contextual Embeddings

### BERT (Contextual Embedding)

In [10]:
from transformers import BertTokenizer, BertModel

# Tải model BERT và tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize và lấy embedding cho một câu
inputs = tokenizer("I like apple", return_tensors="pt")
outputs = model(**inputs)
print(outputs.last_hidden_state)


Downloading: 100%|███████████| 226k/226k [00:00<00:00, 474kB/s]
Downloading: 100%|██████████████████| 48.0/48.0 [00:00<?, ?B/s]
Downloading: 100%|████████████████████| 570/570 [00:00<?, ?B/s]
Downloading: 100%|██████████| 420M/420M [03:55<00:00, 1.87MB/s]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identi

tensor([[[ 0.0134,  0.2599, -0.1307,  ..., -0.0976,  0.2935,  0.2728],
         [ 0.0223,  0.1650, -0.1894,  ...,  0.0197,  0.6733,  0.0629],
         [ 0.4134,  0.4111,  1.1386,  ...,  0.2770,  0.2656, -0.0869],
         [-0.4583, -0.0683, -0.7956,  ...,  0.9551,  0.2224, -0.7772],
         [ 0.8948,  0.1263, -0.2075,  ...,  0.0302, -0.5745, -0.4520]]],
       grad_fn=<NativeLayerNormBackward0>)


### SBERT (Sentence Embeddings)

In [32]:
from sentence_transformers import SentenceTransformer

# Tạo model SBERT
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Câu ví dụ
sentences = ["táo thơm ớt", "chuối táo thơm"]

# Lấy embeddings của các câu
embeddings = model.encode(sentences)

# In embeddings
for sentence, embedding in zip(sentences, embeddings):
    print(f"Sentence: {sentence}")
    print(f"Embedding: {embedding[:5]}...")  # In ra 5 giá trị đầu của embedding


ImportError: cannot import name 'OfflineModeIsEnabled' from 'huggingface_hub.utils' (C:\Users\Hi\miniconda3\envs\gensim_env\lib\site-packages\huggingface_hub\utils\__init__.py)

### XLM-RoBERTa

In [12]:
from transformers import XLMRobertaTokenizer, XLMRobertaModel
import torch

# Tải mô hình và tokenizer cho XLM-RoBERTa
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaModel.from_pretrained('xlm-roberta-base')

# Câu ví dụ
sentence = "táo thơm ớt"

# Tokenize câu và chuyển đổi thành tensor
inputs = tokenizer(sentence, return_tensors="pt")

# Lấy embeddings từ mô hình
outputs = model(**inputs)

# Lấy embedding của từ (mỗi từ có một vector embedding)
# outputs['last_hidden_state'] là một tensor có kích thước [batch_size, sequence_length, hidden_size]
embeddings = outputs.last_hidden_state

# In ra embedding của các từ trong câu (chúng ta có thể lấy embedding của từng token)
for token, embedding in zip(inputs['input_ids'][0], embeddings[0]):
    print(f"Token: {tokenizer.decode([token])}, Embedding: {embedding[:5]}...")  # In ra 5 giá trị đầu của embedding


Downloading: 100%|████████| 4.83M/4.83M [00:03<00:00, 1.67MB/s]
Downloading: 100%|██████████| 25.0/25.0 [00:00<00:00, 8.19kB/s]
Downloading: 100%|█████████████| 615/615 [00:00<00:00, 258kB/s]
Downloading: 100%|████████| 1.04G/1.04G [10:11<00:00, 1.82MB/s]
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Token: <s>, Embedding: tensor([ 0.0634,  0.0917,  0.0492, -0.0073,  0.0354], grad_fn=<SliceBackward0>)...
Token: tá, Embedding: tensor([-0.0240,  0.2000, -0.0731,  0.1386,  0.0661], grad_fn=<SliceBackward0>)...
Token: o, Embedding: tensor([-0.0160,  0.0255,  0.0096, -0.0118,  0.2884], grad_fn=<SliceBackward0>)...
Token: thơm, Embedding: tensor([-0.0179, -0.0102, -0.0108,  0.0245,  0.0115], grad_fn=<SliceBackward0>)...
Token: , Embedding: tensor([ 0.0986,  0.0235, -0.0090,  0.1526,  0.1426], grad_fn=<SliceBackward0>)...
Token: ớt, Embedding: tensor([ 0.0168, -0.0036, -0.0270,  0.1180,  0.0015], grad_fn=<SliceBackward0>)...
Token: </s>, Embedding: tensor([ 0.0525,  0.0826,  0.0079, -0.0108,  0.0513], grad_fn=<SliceBackward0>)...


### PhoBERT 

In [30]:
import torch
from transformers import AutoModel, AutoTokenizer

# Tải mô hình PhoBERT và tokenizer
phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

# Văn bản ví dụ (lưu ý: câu đã được phân tách từ)
sentence = 'Chúng_tôi là những nghiên_cứu_viên .'

# Encode câu bằng tokenizer
input_ids = torch.tensor([tokenizer.encode(sentence)])

# Lấy embeddings từ PhoBERT
with torch.no_grad():
    features = phobert(input_ids)  # Mô hình trả về tuple

# Tính embeddings từ lớp hidden state
# `features` là một tuple gồm (last_hidden_state, pooler_output)
# Chúng ta sẽ lấy `last_hidden_state` để có embeddings cho từng từ
embeddings = features[0]  # Lớp embeddings của từ trong câu

# In embeddings của từ đầu tiên trong câu (embedding của token đầu tiên)
print(f"Embedding của từ đầu tiên: {embeddings[0][0]}")  # In ra 1st token's embedding


RuntimeError: Failed to import transformers.models.auto.modeling_auto because of the following error (look up to see its traceback):
cannot import name '_BaseAutoBackboneClass' from 'transformers.models.auto.auto_factory' (C:\Users\Hi\miniconda3\envs\gensim_env\lib\site-packages\transformers\models\auto\auto_factory.py)